<a href="https://colab.research.google.com/github/arindambits/Advisor-Assist/blob/main/Advisor_Assist_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Multi-Agent Advisor Assist Framework with LangChain + Gemini

# Step 1: Install Dependencies
!pip install langchain langchain-community langchain-google-genai google-generativeai faiss-cpu --quiet


from google.colab import drive
drive.mount('/content/drive')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
Mounted at /content/drive


# Multi-Agent Financial Advisor Assistant

This notebook demonstrates a multi-agent framework for financial advisory services using LangChain and Google's Gemini Pro model. The system uses a ReAct (Reasoning and Acting) agent to interact with a suite of tools, each designed to handle a specific aspect of financial advising.

In [ ]:
# Step 2: Setup Google Gemini LLM and Embeddings
import os
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# llm = ChatGoogleGenerativeAI(model="gemini-pro")
# llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.7)
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

###Create a Lightweight DB

We are using SQLite3 for teh same


In [ ]:

import sqlite3
import pandas as pd

# from google.colab import drive
# drive.mount('/content/drive')

pd.set_option("display.max_columns", None)        # show all columns
pd.set_option("display.expand_frame_repr", False) # prevent wrapping
pd.set_option("display.max_colwidth", None)       # show full text in columns

db_path = '/content/drive/MyDrive/Colab Notebooks/Supporting_Files/Database/customer_data.db'


# Connect to the uploaded database
conn = sqlite3.connect(db_path)

cursor = conn.cursor()

# Execute the create_tables.sql script
with open('/content/drive/MyDrive/Colab Notebooks/Supporting_Files/Database/advisor_assist_schema_Creation.sql', 'r') as f:
   cursor.executescript(f.read())

# Execute the insert_data.sql script
with open('/content/drive/MyDrive/Colab Notebooks/Supporting_Files/Database/advisor_assist_insertscript.sql', 'r') as f:
 cursor.executescript(f.read())


# Dictionary of table names and their respective variable names
tables = {
    "PersonalDemographics": "demographics_df",
    "FinancialTransactionDataView": "transactions_df",
    "ProductAndServiceUsage": "usage_df",
    "clients": "clients_df",
    "portfolios": "portfolios_df",
    "risk_profiles": "risk_profiles_df",
    "goal_recommendations": "goal_recommendations_df",
    "portfolio_analysis_summary": "portfolio_analysis_summary_df"
}

# Loop to query and print each table
for table_name, var_name in tables.items():
    print(f"\n--- Table: {table_name} ---")
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    # print(df)

conn.close()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

--- Table: PersonalDemographics ---
  CustomerID FirstName   LastName DateOfBirth  Gender MaritalStatus EducationLevel EmploymentStatus  AnnualIncome GeographicLocation
0       C001      John        Doe  1976-09-19    Male       Married       Graduate         Employed      120000.0                USA
1       C002      Ajay     Sharma  1986-09-28    Male        Single   Postgraduate    Self-Employed       95000.0              India
2       C003     Maria  Fernandez  1993-11-03  Female        Single       Graduate         Employed       70000.0              India

--- Table: FinancialTransactionDataView ---
    TransactionID client_id TransactionDate TransactionType    Amount TransactionCategory  PaymentMethod  MerchantDetails
0             101      C002      2024-06-17            Sold  18417.82              Equity  Bank Transfer          Zerodha
1            

In [ ]:
def data_query_agent(conn, customer_id):
    tables = ["PersonalDemographics", "clients", "portfolios", "FinancialTransactionDataView"]

    # print("\n--- Table Schemas ---")
    # for table in tables:
    #     schema_df = pd.read_sql_query(f"PRAGMA table_info({table});", conn)
    #     print(f"\nSchema for {table}:\n", schema_df.to_string(index=False))

    # Run the actual joined query
    query = """
    SELECT
        pd.CustomerID, pd.FirstName, pd.LastName, pd.DateOfBirth, pd.Gender,
        pd.MaritalStatus, pd.EducationLevel, pd.EmploymentStatus, pd.AnnualIncome,
        pd.GeographicLocation,
        c.age, c.income, c.net_worth, c.risk_tolerance_score, c.segment,
        p.asset_class, p.holding_value, p.allocation_pct
    FROM PersonalDemographics pd
    JOIN clients c ON pd.CustomerID = c.client_id
    LEFT JOIN portfolios p ON c.client_id = p.client_id
    WHERE pd.CustomerID = ?
    """

    # query = """
    # SELECT
    #     pd.CustomerID, pd.FirstName, pd.LastName, pd.DateOfBirth, pd.Gender,
    #     pd.MaritalStatus, pd.EducationLevel, pd.EmploymentStatus, pd.AnnualIncome,
    #     pd.GeographicLocation,
    #     c.age, c.income, c.net_worth, c.risk_tolerance_score, c.segment,
    #     p.asset_class, p.holding_value, p.allocation_pct,
    #     f.TransactionType, f.Amount, f.TransactionCategory, f.TransactionDate
    # FROM PersonalDemographics pd
    # JOIN clients c ON pd.CustomerID = c.client_id
    # LEFT JOIN portfolios p ON c.client_id = p.client_id
    # LEFT JOIN FinancialTransactionDataView f ON c.client_id = f.client_id
    # WHERE pd.CustomerID = ?
    # """

    df = pd.read_sql_query(query, conn, params=(customer_id,))
    return df


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# This line was changed in the prompt
# You are a senior financial analyst tasked with preparing a profile of a customer
# based on their demographic, financial, investment, and transaction data.

profile_prompt = PromptTemplate(
    input_variables=["customer_data"],
    template="""
You are a senior financial analyst tasked with preparing a profile of a customer
based on their demographic, financial, investment.

Create a clear and descriptive profile that includes:
- Personal Demographics (name, age, education, employment, location)
- Risk and Segment Classification
- Investment Holdings Summary

Only use the data below and structure the summary in paragraphs.

Raw Customer Data:
{customer_data}

Client Profile:
"""

)

print("\n--- Profile Prompt Template ---\n")
print(profile_prompt.template)

profiling_chain = LLMChain(llm=llm, prompt=profile_prompt)

def customer_profiling_agent(df):
    customer_data_str = df.to_string(index=False)
    # print("\n--- Customer Data ---\n")
    # print(customer_data_str)
    return profiling_chain.run(customer_data=customer_data_str)



--- Profile Prompt Template ---


You are a senior financial analyst tasked with preparing a profile of a customer
based on their demographic, financial, investment.

Create a clear and descriptive profile that includes:
- Personal Demographics (name, age, education, employment, location)
- Risk and Segment Classification
- Investment Holdings Summary

Only use the data below and structure the summary in paragraphs.

Raw Customer Data:
{customer_data}

Client Profile:



### Step 3: Initialize FAISS In-Memory Vector Store

We'll use a FAISS vector store to quickly search and retrieve client information. The sample documents below represent the kind of data a financial advisor might have on their clients.

In [ ]:
# Step 3: Initialize FAISS In-Memory Vector Store
from langchain.vectorstores import FAISS
from langchain.schema import Document

# Connect to your database
conn = sqlite3.connect(db_path)  # or your existing in-memory or disk DB
sample_docs = []
customer_ids = ['C001', 'C002', 'C003']
# customer_ids = ['C001']

print("\n--- Generating Client Profiles ---")
for customer_id in customer_ids:
    # print(f"\nCustomer ID : ", customer_id)

    # customer_id = 'C001'
    df = data_query_agent(conn, customer_id)
    print("Raw Query Result:\n", df)

    # Run Gemini 1.5 based profiling agent
    profile_summary = customer_profiling_agent(df)
    # print("\n Customer Profile Summary:\n")
    # print(profile_summary)

    # Append a new Document with the profile summary to sample_docs
    sample_docs.append(Document(page_content=profile_summary))

# sample_docs = [
#     Document(page_content="Client John is a 45-year-old with high risk tolerance and a goal to retire by 55."),
#     Document(page_content="Client Maria, age 32, is conservative and saving for a house."),
#     Document(page_content="Client Ajay frequently discusses market volatility and shows moderate emotional concern."),
#     Document(page_content="Client John holds a diversified portfolio with 60% equities, 30% bonds, 10% crypto."),
#     Document(page_content="Client Maria prefers fixed deposits and low-volatility mutual funds."),
#     Document(page_content="Client Ajay aims to buy a second home and wants to minimize capital gains tax.")
# ]

print("\n ------  Sample Docs Data:\n")
for doc in sample_docs:
  print(doc.page_content)

vector_store = FAISS.from_documents(sample_docs, embedding_model)


--- Generating Client Profiles ---
Raw Query Result:
   CustomerID FirstName LastName DateOfBirth Gender MaritalStatus EducationLevel EmploymentStatus  AnnualIncome GeographicLocation  age    income  net_worth  risk_tolerance_score        segment asset_class  holding_value  allocation_pct
0       C001      John      Doe  1976-09-19   Male       Married       Graduate         Employed      120000.0                USA   45  120000.0   800000.0                     9  Mass Affluent      Equity       480000.0            60.0
1       C001      John      Doe  1976-09-19   Male       Married       Graduate         Employed      120000.0                USA   45  120000.0   800000.0                     9  Mass Affluent        Bond       240000.0            30.0
2       C001      John      Doe  1976-09-19   Male       Married       Graduate         Employed      120000.0                USA   45  120000.0   800000.0                     9  Mass Affluent      Crypto        80000.0            10.0
R

### Step 4: Define Tools (Agents)

Each function below acts as a specialized "tool" or "agent" that the main ReAct agent can use to perform specific tasks.

In [ ]:
# Step 4: Define Tools (Agents)
stored_notes = []
k_value = 2
def search_client_profile(query: str):
    """Searches the vector store for client information."""
    results = vector_store.similarity_search(query, k=k_value)
    return "\n".join([doc.page_content for doc in results])

def record_client_notes(note: str):
    """Records a new note about the client."""
    stored_notes.append(note)
    return "Note recorded."

def generate_persona(_):
    """Generates a client persona based on the recorded notes."""
    return "\n".join([f"- {note}" for note in stored_notes]) or "No notes to summarize."

def analyze_portfolio(_):
    """Analyzes the client's current portfolio."""
    return "Client's portfolio is moderately diversified. Recommend periodic rebalancing."

def evaluate_risk(_):
    """Evaluates the client's risk tolerance."""
    return "Based on behavior and holdings, client has moderate risk exposure."

def recommend_goals(_):
    """Recommends financial goals for the client."""
    return "Suggest increasing equity exposure for early retirement and starting SIP for goal planning."

### Step 5: Custom Prompt Templates per Tool

We create custom prompt templates to guide each specialized agent. This allows us to tailor the instructions for each tool.

In [ ]:
# Step 5: Custom Prompt Templates per Tool
from langchain.prompts import PromptTemplate

base_prompt = PromptTemplate(
    input_variables=["input"],
    template="""
You are a financial advisory assistant. Your job is to take actions using tools to build a client profile.
Input: {input}
Think step by step and choose one tool at a time to make progress.
Respond with Thought, Action, and Action Input.
"""
)

portfolio_prompt = PromptTemplate(
    input_variables=["input"],
    template="""
As a Portfolio Analyzer Agent, review client's asset allocation and diversification based on available profile.
Input: {input}
Respond with Thought, Action, and Action Input.
"""
)

risk_prompt = PromptTemplate(
    input_variables=["input"],
    template="""
You are a Risk Evaluation Agent. Analyze behavioral data and portfolio exposure to assign a risk profile.
Input: {input}
Respond with Thought, Action, and Action Input.
"""
)

goal_prompt = PromptTemplate(
    input_variables=["input"],
    template="""
You are a Goal Recommendation Agent. Use financial goals and profile to suggest strategies.
Input: {input}
Respond with Thought, Action, and Action Input.
"""
)

### ✅ Step 6: Register LangChain Tools

Now, we register the functions we defined as tools that the LangChain agent can use.

In [ ]:
# ✅ Step 6: Register LangChain Tools
from langchain.agents import Tool

tools = [
    Tool(name="SearchClientProfile", func=search_client_profile, description="Search client info using query"),
    Tool(name="RecordNotes", func=record_client_notes, description="Record extracted insights or observations"),
    Tool(name="GeneratePersona", func=generate_persona, description="Generate evolving client persona from notes"),
    Tool(name="AnalyzePortfolio", func=analyze_portfolio, description="Analyze client's current portfolio holdings"),
    Tool(name="EvaluateRisk", func=evaluate_risk, description="Evaluate client's risk tolerance and exposure"),
    Tool(name="RecommendGoals", func=recommend_goals, description="Suggest goals based on client profile and needs")
]

### ✅ Step 7: Initialize Multi-Agent ReAct Loop

We initialize the main ReAct agent. This agent will use the tools we've defined to reason and act on the user's query. We also add a memory component to retain conversation history.

In [ ]:
# ✅ Step 7: Initialize Multi-Agent ReAct Loop
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    agent_kwargs={"prefix": base_prompt.template},
    verbose=True
)


/tmp/ipython-input-1388368342.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")
/tmp/ipython-input-1388368342.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


### ✅ Step 8: Run the Agent

Now we can run the agent with a specific query. The agent will determine which tools to use and in what order to best answer the query.

In [ ]:
# ✅ Step 8: Run the Agent
# query = "Search client John’s profile, record insights, analyze portfolio, evaluate risk, recommend goals and generate a full persona."
query = "Search client Ajay's profile, record insights, analyze portfolio, evaluate risk, recommend goals and generate a full persona."
agent.run(query)



> Entering new AgentExecutor chain...
Thought: First, I need to find Ajay's profile to gather basic information.

Action: SearchClientProfile

Action Input: query = "Ajay"
Observation: Ajay Sharma is a 38-year-old single male residing in India.  He holds a postgraduate degree and is self-employed, earning an annual income of ₹95,000.  His net worth is ₹600,000, placing him in the Upper Middle Class segment.  His risk tolerance score of 6 indicates a moderate risk appetite.

Ajay's investment portfolio is diversified across Equity, Bonds, and Real Estate, with a 60% allocation to Equity, 30% to Bonds, and 10% to Real Estate. His total holdings are valued at ₹600,000.  The data reveals recent active trading activity.  In June 2024 alone, he executed multiple buy and sell transactions across various asset classes.  This includes buying and selling Fixed Deposits, Bonds, and Equity. Notably, he also engaged in buying and selling Real Estate investments, suggesting a potentially active ap

'Ajay Sharma is a 38-year-old single, self-employed male from India with a moderate risk tolerance (score 6). He has an annual income of ₹95,000 and a net worth of ₹600,000, placing him in the Upper Middle Class segment. His portfolio, valued at ₹600,000, is moderately diversified across Equity (60%), Bonds (30%), and Real Estate (10%).  He actively trades across asset classes, suggesting a proactive investment strategy, but further analysis is needed to evaluate its effectiveness.  His moderate risk exposure aligns with his moderate risk tolerance.  Recommended goals include increasing equity exposure for early retirement and starting a Systematic Investment Plan (SIP) for goal-based planning.  Periodic portfolio rebalancing is also recommended.'

### ✅ Optional: Print Notes

Finally, we can print the notes that were recorded during the agent's execution.

In [ ]:
# ✅ Optional: Print Notes
print("\n\n🔸 Final Notes:", stored_notes)



🔸 Final Notes: ['note="Ajay expresses moderate concern about market volatility.  Goals include purchasing a second home and minimizing capital gains tax.']
